In [2]:
import os

import numpy as np
import pandas as pd
import json
import cv2
from sklearn.model_selection import train_test_split
import cv2

# tf imports
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras import backend as K

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    print(e)

1 Physical GPUs, 1 Logical GPU


loading images with labels

In [4]:
f = open("..\data\measurements.json")
data_json = json.loads(f.read())["Measurements"]

vs = data_json.values()

df = pd.json_normalize(vs)

df.head()


,Variety,RGB_Image,Depth_Information,FreshWeightShoot,DryWeightShoot,Height,Diameter,LeafArea
0,Lugano,RGB_27.png,Depth_27.png,5.5,0.42,9.0,11.0,153.9
1,Lugano,RGB_79.png,Depth_79.png,30.3,1.92,8.8,20.2,582.3
2,Lugano,RGB_140.png,Depth_140.png,60.9,2.83,11.6,19.0,960.6
3,Lugano,RGB_203.png,Depth_203.png,112.0,5.76,11.0,22.0,1614.9
4,Lugano,RGB_292.png,Depth_292.png,372.6,14.17,17.0,32.0,3839.6


In [5]:
import albumentations as alb

def augment(train_set, train_labels, iterations):
    # Define a list of image augmentation transformations to apply
    aug_transforms = [
        alb.HorizontalFlip(),
        alb.VerticalFlip(),
        alb.RandomRotate90(),
        alb.Transpose(),
        alb.RandomBrightnessContrast()
    ]

    # Use the Compose function to apply the transformations
    aug = alb.Compose(aug_transforms)

    # Set the number of times to augment the images
    num_augmentations = iterations

    # Iterate through the number of augmentations
    for i in range(num_augmentations):
      # Iterate through your training set and apply the augmentations to each image
      augmented_train_set = []
      augmented_train_labels = []
      for image, label in zip(train_set, train_labels):
        augmented_image = aug(image=image)['image']
        augmented_train_set.append(augmented_image)
        augmented_train_labels.append(label)

      # Add the augmented images and labels to the training set
      train_set = np.concatenate((train_set, augmented_train_set))
      train_labels = np.concatenate((train_labels, augmented_train_labels))

    # Shuffle the training set and labels
    p = np.random.permutation(len(train_set))
    X = train_set[p]
    y = train_labels[p]
    return X, y


In [6]:
"""
    number: depth=0, RGB=1, depth/RGB=2, augment_data: amount of times for data to be augmented
"""
def prepare_data(number=0, y_key='Height', augment_data=0):

    all_images = []
    y = []

    if number == 2:
        path_RGB = '../data/RGBImagesResize/'
        key_RGB = 'RGB_Image'
        flag_RGB = cv2.IMREAD_COLOR

        path_depth = '../data/DepthImagesResize/'
        key_depth = 'Depth_Information'
        flag_depth = cv2.IMREAD_GRAYSCALE

        for image_path_RGB in os.listdir(path_RGB):
            if df.loc[df[key_RGB] == image_path_RGB][y_key].size > 0:
                img_ID = image_path_RGB.strip(".png").strip("RGB_")
                y.append(df.loc[df[key_RGB] == image_path_RGB][y_key].item())
                img_RGB = cv2.imread(path_RGB + image_path_RGB, flag_RGB)
                img_RGB = cv2.resize(img_RGB, (250, 250))
                image_path_depth = path_depth + "Depth_"+ img_ID + ".png"
                img_depth = cv2.imread(image_path_depth, flag_depth)
                img_depth = cv2.resize(img_depth, (250,250))
                img_combined = np.dstack((img_RGB, img_depth))
                all_images.append(img_combined)
    else:
        if number == 1:
            path = '../data/RGBImagesResize/'
            key = 'RGB_Image'
            flag = cv2.IMREAD_COLOR
        else:
            path = '../data/DepthImagesResize/'
            key = 'Depth_Information'
            flag = cv2.IMREAD_GRAYSCALE

        for image_path in os.listdir(path):
            if df.loc[df[key] == image_path][y_key].size > 0:
                y.append(df.loc[df[key] == image_path][y_key].item())
                img = cv2.imread(path + image_path, flag)
                img = cv2.resize(img, (250, 250))
                all_images.append(img)


    X_train, X_test, y_train, y_test = train_test_split(np.array(all_images), y, test_size=0.25, shuffle=True)

    if augment_data > 0:
        X_train, y_train = augment(X_train, y_train, augment_data)

    X_train = tf.convert_to_tensor(X_train)
    X_test = tf.convert_to_tensor(X_test)
    y_train = tf.convert_to_tensor(y_train)
    y_test = tf.convert_to_tensor(y_test)

    return X_train, y_train, X_test, y_test

ResNet Implementation for regression: depth images height

https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba

!!LET OP NIET RUNNEN ZONDER GPU!!

In [7]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, \
    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

In [8]:
def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

In [9]:
def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides=(1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

In [10]:
def create_res_net(input_shape, regression):
    inputs = Input(shape=input_shape)
    num_filters = 64

    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)

    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j == 0 and i != 0), filters=num_filters)
        num_filters *= 2

    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(19, activation='softmax')(t)

    if regression:
        outputs = Dense(1, activation='linear')(t)

    model = Model(inputs, outputs)

    if regression:
        model.compile(
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_absolute_percentage_error']
        )
    else:
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

    return model

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
import datetime
import os

def fit_model(model, epochs, batch_size, X_train, y_train, X_test, y_test):
    timestr = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    name = 'res_net'+timestr

    checkpoint_path = "checkpoints/"+name+"/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    os.system('mkdir {}'.format(checkpoint_dir))

    cp_callback = ModelCheckpoint(
        filepath=checkpoint_path,
        verbose=0,
        save_best_only=True,
        monitor='val_loss',
        mode='min'
    )
    tensorboard_callback = TensorBoard(
        log_dir=f'tensorboard_logs/{name}',
        histogram_freq=1
    )
    # early_stop_callback = EarlyStopping(
    #     monitor='val_loss',
    #     mode='min',
    #     patience=10
    # )

    model.fit(
        x=X_train,
        y=y_train,
        epochs=epochs,
        verbose=1,
        validation_data=(X_test, y_test),
        batch_size=batch_size,
        callbacks=[cp_callback, tensorboard_callback]
    )

In [12]:
model = create_res_net((250,250,3),True)
X_train, y_train, X_test, y_test = prepare_data(1, 'Diameter', 0)

In [14]:
fit_model(model, 100, 20, X_train, y_train, X_test, y_test)

Epoch 1/100
15/15 [==============================] - ETA: 0s - loss: 160.4766 - mean_absolute_percentage_error: 40.2802

INFO:tensorflow:Assets written to: checkpoints/res_net20221220-135423\cp-0001.ckpt\assets


INFO:tensorflow:Assets written to: checkpoints/res_net20221220-135423\cp-0001.ckpt\assets


15/15 [==============================] - 58s 3s/step - loss: 160.4766 - mean_absolute_percentage_error: 40.2802 - val_loss: 522995416170496.0000 - val_mean_absolute_percentage_error: 103469016.0000
Epoch 2/100
15/15 [==============================] - ETA: 0s - loss: 17.0243 - mean_absolute_percentage_error: 15.1390

INFO:tensorflow:Assets written to: checkpoints/res_net20221220-135423\cp-0002.ckpt\assets


...

INFO:tensorflow:Assets written to: checkpoints/res_net20221220-135423\cp-0026.ckpt\assets


15/15 [==============================] - 33s 2s/step - loss: 3.2976 - mean_absolute_percentage_error: 6.5801 - val_loss: 6.2091 - val_mean_absolute_percentage_error: 8.4151
Epoch 27/100
15/15 [==============================] - 23s 2s/step - loss: 3.3119 - mean_absolute_percentage_error: 6.3910 - val_loss: 9.3124 - val_mean_absolute_percentage_error: 12.4223
Epoch 28/100
15/15 [==============================] - 23s 2s/step - loss: 3.2992 - mean_absolute_percentage_error: 6.6221 - val_loss: 7.5607 - val_mean_absolute_percentage_error: 9.3902
Epoch 29/100
15/15 [==============================] - 23s 2s/step - loss: 3.2630 - mean_absolute_percentage_error: 6.6324 - val_loss: 7.4708 - val_mean_absolute_percentage_error: 9.9915
Epoch 30/100
15/15 [==============================] - ETA: 0s - loss: 3.3876 - mean_absolute_percentage_error: 6.7785

INFO:tensorflow:Assets written to: checkpoints/res_net20221220-135423\cp-0030.ckpt\assets


INFO:tensorflow:Assets written to: checkpoints/res_net20221220-135423\cp-0030.ckpt\assets


15/15 [==============================] - 33s 2s/step - loss: 3.3876 - mean_absolute_percentage_error: 6.7785 - val_loss: 5.9010 - val_mean_absolute_percentage_error: 8.7941
Epoch 31/100
15/15 [==============================] - 23s 2s/step - loss: 3.2336 - mean_absolute_percentage_error: 6.3365 - val_loss: 7.9636 - val_mean_absolute_percentage_error: 10.0576
Epoch 32/100
15/15 [==============================] - 23s 2s/step - loss: 2.9204 - mean_absolute_percentage_error: 6.4825 - val_loss: 7.1068 - val_mean_absolute_percentage_error: 8.8967
Epoch 33/100
15/15 [==============================] - 23s 2s/step - loss: 3.7075 - mean_absolute_percentage_error: 7.2498 - val_loss: 10.3833 - val_mean_absolute_percentage_error: 11.1806
Epoch 34/100
15/15 [==============================] - 23s 2s/step - loss: 3.0928 - mean_absolute_percentage_error: 6.3123 - val_loss: 9.3904 - val_mean_absolute_percentage_error: 10.3927
Epoch 35/100
15/15 [==============================] - 23s 2s/step - loss: 3.521